<a href="https://colab.research.google.com/github/kittisaktaoma/colab_practice/blob/master/Test_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Library 



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!wget -c https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
#!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
#!time bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local
#!time conda install -q -y -c conda-forge rdkit
#import sys
#sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
! pip install keras
! pip install tensorflow
! pip install keras-gcn
! pip install numpy
! pip install matplotlib
! pip install scipy


In [49]:
from rdkit import Chem
import tensorflow as tf
import tensorflow.keras.layers as layers
import pandas as pd

In [47]:
df = pd.read_table('https://raw.githubusercontent.com/kittisaktaoma/colab_practice/master/dataset/toy_dataset.txt', header=0)
df['activity_class'] = pd.factorize( df.ACTIVITY_COMMENT )
pd.factorize( df.ACTIVITY_COMMENT )
len(pd.factorize( df.ACTIVITY_COMMENT ))
df['activity_class'] = pd.factorize( df.ACTIVITY_COMMENT )[0]
 
df.to_csv('./preprocessed_jak3.csv', index=False)

mv: cannot stat 'jak3_chembl.txt': No such file or directory


In [38]:
atom_features = layers.Input(shape=(75,))
degree_slice = layers.Input(shape=(2,), dtype=tf.int32)
membership = layers.Input(shape=tuple(), dtype=tf.int32)

deg_adjs = []
for i in range(0, 10 + 1):
    deg_adj = layers.Input(shape=(i+1,), dtype=tf.int32)
    deg_adjs.append(deg_adj)